In [1]:
# audio data fed into normalized LSTM with Conv layer


# First, import necessary libraries
import tensorflow as tf
import numpy as np
import keras.backend as K

/Users/chloeloughridge/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
Using TensorFlow backend.


In [7]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization, Conv1D
from keras.layers import LSTM
# my custom data_utils file
from data_utils_local08 import *

In [3]:
# uploading the X values
#X_input = load_Xinput(get_fc6_directory(7)) #VGG16 data

# X_input = np.load('../Desktop/InceptionResNetV2_feature_data.npy') #resnet data!

X_input = np.load('../Desktop/audio_feature_data.npy')

print(X_input.shape)

(7, 207, 1583)


In [4]:
# uploading the Y values
y_data_input = np.load('../Desktop/labels.npy')
print(y_data_input.shape)

(7, 219)


In [5]:
timesteps = X_input.shape[1]
data_dim = X_input.shape[2]

X_train = X_input[:4,:timesteps, :]
Y_train = y_data_input[:4, :timesteps]

X_test = X_input[4:, :timesteps, :]
Y_test = y_data_input[4:, :timesteps]

print(X_test.shape, X_train.shape, Y_train.shape, Y_test.shape)

(3, 207, 1583) (4, 207, 1583) (4, 207) (3, 207)


In [8]:
# constructing a many-to-one LSTM model in keras --> inspiration: https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras
# i will start by training a model on only the VGG16 fc6 layer output (that's just one feature)
# should I eventually abstract this LSTM model? Create its own object file?
model = Sequential()

# normalization
model.add(BatchNormalization(input_shape=(timesteps, data_dim)))

model.add(Conv1D(10, kernel_size=3, activation="relu"))

model.add(LSTM(timesteps, return_sequences=True))
#input_shape=(timesteps, data_dim)

# going to try adding a flatten layer in here
model.add(Flatten()) # I got this from a github thing, but I still don't completely understand why it works
# add the final dense layer and then softmax
model.add(Dense(timesteps, activation='sigmoid'))
# going to add a softmax activation to this
#model.add(Activation('softmax'))

In [9]:
# a new metric of evaluation! the F-score!
def FScore2(y_true, y_pred):
    '''
    The F score, beta=2
    '''
    B2 = K.variable(4)
    OnePlusB2 = K.variable(5)
    pred = K.round(y_pred)
    tp = K.sum(K.cast(K.less(K.abs(pred - K.clip(y_true, .5, 1.)), 0.01), 'float32'), -1)
    fp = K.sum(K.cast(K.greater(pred - y_true, 0.1), 'float32'), -1)
    fn = K.sum(K.cast(K.less(pred - y_true, -0.1), 'float32'), -1)

    f2 = OnePlusB2 * tp / (OnePlusB2 * tp + B2 * fn + fp)

    return K.mean(f2)

In [10]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy', FScore2])

In [11]:
# running the LSTM model
model.fit(X_train, Y_train, epochs = 20, batch_size = 128, validation_data=(X_test, Y_test))
print("finished training!")

Train on 4 samples, validate on 3 samples
Epoch 1/20
4/4 [==============================] - 2s 547ms/step - loss: 0.6932 - binary_accuracy: 0.5181 - FScore2: 0.3259 - val_loss: 4.2617 - val_binary_accuracy: 0.6651 - val_FScore2: 0.0398
Epoch 2/20
4/4 [==============================] - 0s 125ms/step - loss: 1.6592 - binary_accuracy: 0.8551 - FScore2: 0.4668 - val_loss: 4.1094 - val_binary_accuracy: 0.7021 - val_FScore2: 0.0204
Epoch 3/20
4/4 [==============================] - 1s 136ms/step - loss: 1.2466 - binary_accuracy: 0.8925 - FScore2: 0.5873 - val_loss: 5.1023 - val_binary_accuracy: 0.6248 - val_FScore2: 0.2922
Epoch 4/20
4/4 [==============================] - 0s 123ms/step - loss: 2.8375 - binary_accuracy: 0.8225 - FScore2: 0.5529 - val_loss: 5.2386 - val_binary_accuracy: 0.6248 - val_FScore2: 0.2922
Epoch 5/20
4/4 [==============================] - 0s 121ms/step - loss: 2.8374 - binary_accuracy: 0.8225 - FScore2: 0.5529 - val_loss: 5.2693 - val_binary_accuracy: 0.6248 - val_FSco

In [29]:
# trying to view the model output
out = model.predict(X_train)

movie_index = 1

print("model prediction:")
print(out[movie_index])
print("target:")
print(Y_train[movie_index])

print("before 64:")
print(out[movie_index][63])
print("64:")
print(out[movie_index][65])

print("rounded")
print(np.round(out)[movie_index])

model prediction:
[1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 1.9221041e-28 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 2.1595651e-19
 1.0000000e+00 1.0000000e+00 2.3602658e-26 1.0000000e+00 1.0000000e+00
 4.4973351e-18 3.1209957e-32 1.9935814e-32 3.3743915e-34 1.0000000e+00
 1.0000000e+00 2.1284409e-12 1.6664713e-14 1.0000000e+00 1.0000000e+00
 1.0000000e+00 1.0000000e+00 1.7663178e-08 1.0000000e+00 1.0000000e+00
 1.0000000e+00 1.0000000e+00 2.7702462e-30 2.5049629e-30 1.0000000e+00
 2.6652263e-31 4.6479136e-31 2.5350656e-34 4.2515438e-35 4.0209436e-32
 1.0000000e+00 9.4842254e-15 1.0000000e+00 1.4375714e-11 1.0000000e+00
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.

In [ ]:
#try visualizing this model at some point?